### クラスデコレータ

In [ ]:
# オブジェクト名を短縮する
def short_repr(cls):
    cls.__repr__ = lambda self: super(cls, self).__repr__()[:8]
    return cls

@short_repr
class ClassWithRelativelyLongName:
    pass

print(ClassWithRelativelyLongName())

In [ ]:
# クロージャを使ってパラメータ化
def parametrized_short_repr(max_width=8):
    """オブジェクト表現を短縮するパラメータ化したデコレータ
    """
    def parametrized(cls):
        """実際のデコレータに相当する内部ラッパー関数
        """
        class ShortlyRepresented(cls):
            """デコレートされた振る舞いをするサブクラス
            """
            def __repr__(self):
                return super().__repr__()[:max_width]
        return ShortlyRepresented
    return parametrized


@parametrized_short_repr(10)
class ClassWithRelativelyLongName:
    pass


print(ClassWithRelativelyLongName())
print(ClassWithRelativelyLongName().__class__)
print(ClassWithRelativelyLongName().__doc__)

#### __new__()を使用してインスタンス生成処理をオーバーライド

In [ ]:
# インスタンスが生成された数を保持。initよりnewが先に呼ばれる
class InstanceCountingClass:
    instances_created = 0
    def __new__(cls, *args, **kwargs):
        print("__new__() called with:", cls, args, kwargs)
        instance = super().__new__(cls)
        instance.number = cls.instances_created
        cls.instances_created += 1
        return instance
    
    def __init__(self, attribute):
        print("__init__() called with:", self, attribute)
        self.attribute = attribute


instance1 = InstanceCountingClass("abc")
instance2 = InstanceCountingClass("xyz")
print(instance1.number, instance1.instances_created)
print(instance2.number, instance2.instances_created)

In [ ]:
# __new__()にて他クラスを返却すると__init__()は呼ばれない
class NonZero(int):
    def __new__(cls, value):
        return super().__new__(cls, value) if value != 0 else None
    
    def __init__(self, skipped_value):
        # __init__は呼ばれるか確認のため実装
        print("__init__() called")
        super().__init__()

print(type(NonZero(-12)))
print(type(NonZero(0)))
print(NonZero(-3.12))